# 导包和环境配置

In [1]:
import findspark
findspark.init()
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf
from pyspark import SQLContext
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import IntegerType
sc = SparkContext("local", "first")
logFile = "file:///F:/FBDP/实验/实验四/logfile1.txt"  
logData = sc.textFile(logFile).cache()
spark = SparkSession.builder.config("spark.driver.memory", "16g").getOrCreate()

# 读入数据并填补缺失值

In [2]:
df=spark.read.options(header='True') .csv("file:///F:/FBDP/实验/实验四/train_data.csv")
df = df.na.fill(-1)
df = df.na.fill('-1')

# 把categorical变量转换为数值(略显粗暴)

In [3]:
#class
class_indexer = StringIndexer(inputCol='class', outputCol='class_num').fit(df)
df = class_indexer.transform(df)
class_onehoter = OneHotEncoder(inputCol='class_num', outputCol='class_vector')
df = class_onehoter.transform(df)
#sub_class
sub_class_indexer = StringIndexer(inputCol='sub_class', outputCol='sub_class_num').fit(df)
df = sub_class_indexer.transform(df)
sub_class_onehoter = OneHotEncoder(inputCol='sub_class_num', outputCol='sub_class_vector')
df = sub_class_onehoter.transform(df)
#work_type
work_type_indexer = StringIndexer(inputCol='work_type', outputCol='work_type_num').fit(df)
df = work_type_indexer.transform(df)
work_type_onehoter = OneHotEncoder(inputCol='work_type_num', outputCol='work_type_vector')
df = work_type_onehoter.transform(df)
#employer_type
employer_type_indexer = StringIndexer(inputCol='employer_type', outputCol='employer_type_num').fit(df)
df = employer_type_indexer.transform(df)
employer_type_onehoter = OneHotEncoder(inputCol='employer_type_num', outputCol='employer_type_vector')
df = employer_type_onehoter.transform(df)
#industry
industry_indexer = StringIndexer(inputCol='industry', outputCol='industry_num').fit(df)
df = industry_indexer.transform(df)
industry_onehoter = OneHotEncoder(inputCol='industry_num', outputCol='industry_vector')
df = industry_onehoter.transform(df)
#work_year
work_year_indexer = StringIndexer(inputCol='work_year', outputCol='work_year_num').fit(df)
df = work_year_indexer.transform(df)
work_year_onehoter = OneHotEncoder(inputCol='work_year_num', outputCol='work_year_vector')
df = work_year_onehoter.transform(df)
#df.show(3)
#issue_date
issue_date_indexer = StringIndexer(inputCol='issue_date', outputCol='issue_date_num').fit(df)
df = issue_date_indexer.transform(df)
issue_date_onehoter = OneHotEncoder(inputCol='issue_date_num', outputCol='issue_date_vector')
df = issue_date_onehoter.transform(df)
#df.show(3)
#earlies_credit_mon
earlies_credit_mon_indexer = StringIndexer(inputCol='earlies_credit_mon', outputCol='earlies_credit_mon_num').fit(df)
df = earlies_credit_mon_indexer.transform(df)
earlies_credit_mon_onehoter = OneHotEncoder(inputCol='earlies_credit_mon_num', outputCol='earlies_credit_mon_vector')
df = earlies_credit_mon_onehoter.transform(df)
df.show(3)

+-------+-------+----------+------------+--------+---------------+-----+---------+---------+-------------+------------------------------+---------+-----------+-----------------+-------------+--------+----------+----------+---+---------+------+---------------+--------------+-----------+------------+----------------+------------+-------------------+------------------------+----------+----------+-------------------+------------------+-----+-----------+---+---+----+----+---+---+----------+---------+-------------+-------------+----------------+-------------+----------------+-----------------+--------------------+------------+---------------+-------------+----------------+--------------+-----------------+----------------------+-------------------------+
|loan_id|user_id|total_loan|year_of_loan|interest|monthly_payment|class|sub_class|work_type|employer_type|                      industry|work_year|house_exist|house_loan_status|censor_status|marriage|offsprings|issue_date|use|post_code|region

# 把string列转为int

In [4]:
tmpCols=['total_loan', 'year_of_loan', 'interest', 'monthly_payment', 'class_vector','sub_class_vector','work_type_vector','employer_type_vector','industry_vector','issue_date_vector','earlies_credit_mon_vector','house_exist','house_loan_status','censor_status','marriage','offsprings','use','post_code','region','debt_loan_ratio','del_in_18month','scoring_low','scoring_high','pub_dero_bankrup','early_return','early_return_amount','early_return_amount_3mon','recircle_b','recircle_u','initial_list_status','title','policy_code','f0','f1','f2','f3','f4','f5']
for i in tmpCols:
    if "vector" in i:
        print("")
    else:
        df = df.withColumn(i, df[i].cast('double'))
#df = df.withColumn("total_loan", df["total_loan"].cast(IntegerType()))
df = df.withColumn('is_default', df['is_default'].cast(IntegerType()))
df.show(3)








+-------+-------+----------+------------+--------+---------------+-----+---------+---------+-------------+------------------------------+---------+-----------+-----------------+-------------+--------+----------+----------+---+---------+------+---------------+--------------+-----------+------------+----------------+------------+-------------------+------------------------+----------+----------+-------------------+------------------+-----+-----------+----+----+----+----+----+----+----------+---------+-------------+-------------+----------------+-------------+----------------+-----------------+--------------------+------------+---------------+-------------+----------------+--------------+-----------------+----------------------+-------------------------+
|loan_id|user_id|total_loan|year_of_loan|interest|monthly_payment|class|sub_class|work_type|employer_type|                      industry|work_year|house_exist|house_loan_status|censor_status|marriage|offsprings|issue_date|use|post_

# 把输入特征合并到一列

In [5]:
data = df.drop('is_default')
feas = data.columns
df_assembler = VectorAssembler(inputCols=['total_loan', 'year_of_loan', 'interest', 'monthly_payment', 'class_vector','sub_class_vector','work_type_vector','work_year_vector','employer_type_vector','industry_vector','issue_date_vector','earlies_credit_mon_vector','house_exist','house_loan_status','censor_status','marriage','offsprings','use','post_code','region','debt_loan_ratio','del_in_18month','scoring_low','scoring_high','pub_dero_bankrup','early_return','early_return_amount','early_return_amount_3mon','recircle_b','recircle_u','initial_list_status','title','policy_code','f0','f1','f2','f3','f4','f5'],outputCol='features')
print(df_assembler)
data = df_assembler.transform(df)
data.show()

VectorAssembler_4d1f75ce8f25
+-------+-------+----------+------------+--------+---------------+-----+---------+---------+--------------+------------------------------+---------+-----------+-----------------+-------------+--------+----------+----------+---+---------+------+---------------+--------------+-----------+------------+----------------+------------+-------------------+------------------------+----------+----------+-------------------+------------------+-------+-----------+----+----+----+----+----+----+----------+---------+-------------+-------------+----------------+-------------+----------------+-----------------+--------------------+------------+---------------+-------------+----------------+--------------+-----------------+----------------------+-------------------------+--------------------+
|loan_id|user_id|total_loan|year_of_loan|interest|monthly_payment|class|sub_class|work_type| employer_type|                      industry|work_year|house_exist|house_loan_status|censor_

# 划分数据集（8：2）

In [6]:
data_set = data.select(['features', 'is_default'])
train_df, test_df = data_set.randomSplit([0.8, 0.2])
#print(' train_df shape : (%d , %d)'%(train_df.count(), len(train_df.columns)))
#print(' test_df  shape: :(%d , %d)'%(test_df.count(), len(test_df.columns)))

# 训练逻辑回归模型

In [7]:
log_reg = LogisticRegression(labelCol = 'is_default').fit(train_df)
train_pred = log_reg.evaluate(train_df).predictions
train_pred.filter(train_pred['is_default'] == 1).filter(train_pred['prediction'] == 1).select(['is_default', 'prediction', 'probability']).show(10, False)


+----------+----------+----------------------------------------+
|is_default|prediction|probability                             |
+----------+----------+----------------------------------------+
|1         |1.0       |[0.4515515604299872,0.5484484395700129] |
|1         |1.0       |[0.4954833439818917,0.5045166560181082] |
|1         |1.0       |[0.4983561114194535,0.5016438885805465] |
|1         |1.0       |[0.4897249834257105,0.5102750165742895] |
|1         |1.0       |[0.3807023294967067,0.6192976705032932] |
|1         |1.0       |[0.4187524603543903,0.5812475396456096] |
|1         |1.0       |[0.4638689563627991,0.5361310436372009] |
|1         |1.0       |[0.48887976709352776,0.5111202329064723]|
|1         |1.0       |[0.4209914147885416,0.5790085852114585] |
|1         |1.0       |[0.4760275913831084,0.5239724086168916] |
+----------+----------+----------------------------------------+
only showing top 10 rows



# 评估模型

In [8]:
test_result = log_reg.evaluate(test_df).predictions
test_result.show(3)

+--------------------+----------+--------------------+--------------------+----------+
|            features|is_default|       rawPrediction|         probability|prediction|
+--------------------+----------+--------------------+--------------------+----------+
|(926,[0,1,2,3,4,1...|         1|[1.54895071058989...|[0.82476213017558...|       0.0|
|(926,[0,1,2,3,4,1...|         1|[0.69838563314801...|[0.66782974912336...|       0.0|
|(926,[0,1,2,3,4,1...|         0|[2.72567166437477...|[0.93852458219740...|       0.0|
+--------------------+----------+--------------------+--------------------+----------+
only showing top 3 rows



# 模型accuracy

In [9]:
tp = test_result[(test_result.is_default == 1) & (test_result.prediction == 1)].count()
tn = test_result[(test_result.is_default == 0) & (test_result.prediction == 1)].count()
fp = test_result[(test_result.is_default == 0) & (test_result.prediction == 1)].count()
fn = test_result[(test_result.is_default == 1) & (test_result.prediction == 0)].count()
# Accuracy

print('test accuracy is : %f'%((tp+tn)/(tp+tn+fp+fn)))


test accuracy is : 0.444605


# 模型召回率、准确率、f1 score

In [10]:
recal=tp/(tp+fn)
prec=tp/(tp+fp)
print('test recall is : %f'%(recal))
print('test precision is : %f'%(prec))
print('test f1-score is : %f'%(2*recal*prec/(prec+recal)))

test recall is : 0.420291
test precision is : 0.656961
test f1-score is : 0.512628
